In [39]:
import numpy as np
import pandas as pd

In [40]:
categories = pd.read_csv('../../data_q2/q2-ucsd-cat-map.csv')
consumer = pd.read_parquet('../../data_q2/q2-ucsd-consDF.pqt')
acct = pd.read_parquet('../../data_q2/q2-ucsd-acctDF.pqt')
transactions = pd.read_parquet('../../data_q2/q2-ucsd-trxnDF.pqt')

In [41]:
# categories = pd.read_csv('../../data_q2/q2-ucsd-cat-map.csv')
# consumer = pd.read_parquet('../../data_q2/q2-ucsd-consDF.pqt')
# acct = pd.read_parquet("../../data_q2/q2-ucsd-acctIDF.pqt")
# transactions = pd.read_parquet("../../data_q2/q2-ucsd-trxnDF.pqt")

In [42]:
consumer.head()

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET
0,0,2021-09-01,726.0,0.0
1,1,2021-07-01,626.0,0.0
2,2,2021-05-01,680.0,0.0
3,3,2021-03-01,734.0,0.0
4,4,2021-10-01,676.0,0.0


In [43]:
# consumer.groupby('DQ_TARGET').count() #10994 0.0 vs 1006 1.0


In [44]:
transactions.head()

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16


In [84]:
categories

,category_id,category
0,0,SELF_TRANSFER
1,1,EXTERNAL_TRANSFER
2,2,DEPOSIT
3,3,PAYCHECK
4,4,MISCELLANEOUS
5,5,PAYCHECK_PLACEHOLDER
6,6,REFUND
7,7,INVESTMENT_INCOME
8,8,OTHER_BENEFITS
9,9,UNEMPLOYMENT_BENEFITS


In [46]:
acct.head()

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90


In [47]:
len(acct['prism_consumer_id'].unique())

13009

In [48]:
# merge 
acct['account_type'].unique()

# most important account_types: ['CHECKING', 'SAVINGS', 'CREDIT CARD', 'LOAN]
# most_important_accounts = ['CHECKING', 'SAVINGS', 'CREDIT CARD', 'LOAN']
acctDF = acct.copy()
total_balance = acctDF.groupby('prism_consumer_id')['balance'].sum()
total_balance

pd.DataFrame(total_balance) #get total balance for each consumer

,balance
prism_consumer_id,
0,320.37
1,3302.42
10,824.24
100,2655.47
1000,95.25
...,...
9995,0.00
9996,252.67
9997,611.28


In [49]:
consumer_balance = consumer.merge(pd.DataFrame(total_balance), on = 'prism_consumer_id', how = 'outer')
consumer_balance['std_credit'] = (consumer_balance['credit_score'] - consumer_balance['credit_score'].mean()) / consumer_balance['credit_score'].std()
consumer_balance["std_balance"] = (
    consumer_balance["balance"] - consumer_balance["balance"].mean()
    ) / consumer_balance["balance"].std()
consumer_balance

,prism_consumer_id,evaluation_date,credit_score,DQ_TARGET,balance,std_credit,std_balance
0,0,2021-09-01,726.0,0.0,320.37,0.846851,-0.146222
1,1,2021-07-01,626.0,0.0,3302.42,-0.459894,-0.090027
2,10,2022-02-01,654.0,0.0,824.24,-0.094005,-0.136727
3,100,2021-12-01,750.0,0.0,2655.47,1.160470,-0.102219
4,1000,2021-03-01,756.0,0.0,95.25,1.238874,-0.150464
...,...,...,...,...,...,...,...
14995,9995,2023-03-02,578.0,NaN,0.00,-1.087132,-0.152259
14996,9996,2023-10-11,610.0,NaN,252.67,-0.668973,-0.147498
14997,9997,2023-05-25,675.0,NaN,611.28,0.180411,-0.140740
14998,9998,2023-11-02,534.0,NaN,-862.99,-1.662099,-0.168522


In [50]:
!pip install xgboost
# !pip uninstall xgboost -y
import xgboost

Python(11869) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [51]:
!pip install lightgbm
import lightgbm as lgb
print(lgb.__version__)

Python(11870) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
4.5.0


In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
import warnings

def run_classification(
    feature_column, target_column, dataset, test_size=0.2, random_state=42
):
    """
    Run Logistic Regression and Random Forest Classification on a dataset.

    Parameters:
        feature_column (list): List of columns that can be used as features.
        target_column (str): The name of the target column.
        dataset (pd.DataFrame): The dataset containing the features and target.
        test_size (float): Proportion of the dataset to include in the test split (default 0.3).
        random_state (int): Random seed for reproducibility (default 42).

    Returns:
        None: Prints the classification reports for both models.
    """
    warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

    # Drop NaN values and shuffle the dataset
    dataset = dataset.dropna()
    dataset = dataset.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Define features and target
    X = dataset[feature_column]
    y = dataset[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # balance the dataset
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    # Logistic Regression
    log_model = LogisticRegression(class_weight = 'balanced')
    log_model.fit(X_train_bal, y_train_bal)
    log_y_pred = log_model.predict(X_test)
    print(f"Logistic Regression for {feature_column}")
    print(classification_report(y_test, log_y_pred))

    # Random Forest Classification
    rfc_model = RandomForestClassifier(random_state=random_state)
    rfc_model.fit(X_train_bal, y_train_bal)
    rfc_y_pred = rfc_model.predict(X_test)
    print(f"\nRandom Forest Classification for {feature_column}")
    print(classification_report(y_test, rfc_y_pred))

    # Light GBM 
    lgb_model = lgb.LGBMClassifier()
    lgb_model.fit(X_train_bal, y_train_bal)
    lgb_y_pred = lgb_model.predict(X_test)
    print(f"\nLGB Model Classification for {feature_column}")
    print(classification_report(y_test, lgb_y_pred))

    # SVC Model
    svc_model = SVC(probability=True, random_state=random_state)
    svc_model.fit(X_train_bal, y_train_bal)
    svc_y_pred = svc_model.predict(X_test)
    print(f"\nSVM Classification for {feature_column}")
    print(classification_report(y_test, svc_y_pred))

    # XGB Model
    xgb_model = xgb.XGBClassifier(
        scale_pos_weight=len(y_train) / y_train.sum()
        )
    xgb_model.fit(X_train_bal, y_train_bal)
    xgb_y_pred = xgb_model.predict(X_test)
    print("\nXGBoost:")
    print(classification_report(y_test, xgb_y_pred))

In [53]:
# Added ROC AUC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import UndefinedMetricWarning
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
import warnings

def run_classification2(
    feature_column, target_column, dataset, test_size=0.2, random_state=42
):
    """
    Run Logistic Regression, Random Forest, LightGBM, SVM, and XGBoost Classification on a dataset.
    Includes ROC-AUC evaluation for all models.

    Parameters:
        feature_column (list): List of columns that can be used as features.
        target_column (str): The name of the target column.
        dataset (pd.DataFrame): The dataset containing the features and target.
        test_size (float): Proportion of the dataset to include in the test split (default 0.2).
        random_state (int): Random seed for reproducibility (default 42).

    Returns:
        None: Prints the classification reports and ROC-AUC scores for all models.
    """
    warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

    # Drop NaN values and shuffle the dataset
    dataset = dataset.dropna()
    dataset = dataset.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Define features and target
    X = dataset[feature_column]
    y = dataset[target_column]

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Balance the dataset using SMOTE
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    # Logistic Regression
    log_model = LogisticRegression(class_weight='balanced')
    log_model.fit(X_train_bal, y_train_bal)
    log_y_pred = log_model.predict(X_test)
    log_y_proba = log_model.predict_proba(X_test)[:, 1]
    print(f"Logistic Regression for {feature_column}")
    print(classification_report(y_test, log_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, log_y_proba):.3f}")

    # Random Forest Classification
    rfc_model = RandomForestClassifier(random_state=random_state)
    rfc_model.fit(X_train_bal, y_train_bal)
    rfc_y_pred = rfc_model.predict(X_test)
    rfc_y_proba = rfc_model.predict_proba(X_test)[:, 1]
    print(f"\nRandom Forest Classification for {feature_column}")
    print(classification_report(y_test, rfc_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, rfc_y_proba):.3f}")

    # LightGBM
    lgb_model = lgb.LGBMClassifier()
    lgb_model.fit(X_train_bal, y_train_bal)
    lgb_y_pred = lgb_model.predict(X_test)
    lgb_y_proba = lgb_model.predict_proba(X_test)[:, 1]
    print(f"\nLightGBM Classification for {feature_column}")
    print(classification_report(y_test, lgb_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, lgb_y_proba):.3f}")

    # SVM Classification
    svc_model = SVC(probability=True, random_state=random_state)
    svc_model.fit(X_train_bal, y_train_bal)
    svc_y_pred = svc_model.predict(X_test)
    svc_y_proba = svc_model.predict_proba(X_test)[:, 1]
    print(f"\nSVM Classification for {feature_column}")
    print(classification_report(y_test, svc_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, svc_y_proba):.3f}")

    # XGBoost Classification
    xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_train) / y_train.sum())
    xgb_model.fit(X_train_bal, y_train_bal)
    xgb_y_pred = xgb_model.predict(X_test)
    xgb_y_proba = xgb_model.predict_proba(X_test)[:, 1]
    print(f"\nXGBoost Classification for {feature_column}")
    print(classification_report(y_test, xgb_y_pred))
    print(f"ROC-AUC Score: {roc_auc_score(y_test, xgb_y_proba):.3f}")


In [54]:
run_classification(["std_credit"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.95      0.74      0.83      1897
         1.0       0.19      0.63      0.29       185

    accuracy                           0.73      2082
   macro avg       0.57      0.68      0.56      2082
weighted avg       0.89      0.73      0.78      2082


Random Forest Classification for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87      1897
         1.0       0.16      0.38      0.23       185

    accuracy                           0.77      2082
   macro avg       0.55      0.60      0.55      2082
weighted avg       0.86      0.77      0.81      2082

[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000548 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bin

In [55]:
# Example usage:
run_classification2(["std_credit"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.95      0.74      0.83      1897
         1.0       0.19      0.63      0.29       185

    accuracy                           0.73      2082
   macro avg       0.57      0.68      0.56      2082
weighted avg       0.89      0.73      0.78      2082

ROC-AUC Score: 0.750

Random Forest Classification for ['std_credit']
              precision    recall  f1-score   support

         0.0       0.93      0.81      0.87      1897
         1.0       0.16      0.38      0.23       185

    accuracy                           0.77      2082
   macro avg       0.55      0.60      0.55      2082
weighted avg       0.86      0.77      0.81      2082

ROC-AUC Score: 0.662
[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_row_wise=true` to remove

In [56]:
# Example usage:  #features, target, dataframe
run_classification(["std_balance"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      1897
         1.0       0.11      0.85      0.20       185

    accuracy                           0.40      2082
   macro avg       0.54      0.61      0.36      2082
weighted avg       0.89      0.40      0.49      2082


Random Forest Classification for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.92      0.68      0.79      1897
         1.0       0.11      0.41      0.18       185

    accuracy                           0.66      2082
   macro avg       0.52      0.55      0.48      2082
weighted avg       0.85      0.66      0.73      2082

[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000530 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total B

In [57]:
run_classification2(["std_balance"], "DQ_TARGET", consumer_balance)

Logistic Regression for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      1897
         1.0       0.11      0.85      0.20       185

    accuracy                           0.40      2082
   macro avg       0.54      0.61      0.36      2082
weighted avg       0.89      0.40      0.49      2082

ROC-AUC Score: 0.701

Random Forest Classification for ['std_balance']
              precision    recall  f1-score   support

         0.0       0.92      0.68      0.79      1897
         1.0       0.11      0.41      0.18       185

    accuracy                           0.66      2082
   macro avg       0.52      0.55      0.48      2082
weighted avg       0.85      0.66      0.73      2082

ROC-AUC Score: 0.589
[LightGBM] [Info] Number of positive: 7596, number of negative: 7596
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000223 seconds.
You can set `force_col_wise=true` to remo

In [113]:
#NEW FEATURES: credit and debit

credit_transactions = transactions[transactions['credit_or_debit'] == 'CREDIT']
debit_transactions = transactions[transactions['credit_or_debit'] == 'DEBIT']

credit_features = credit_transactions.groupby('prism_consumer_id').agg(
    total_credit=('amount', 'sum')
).reset_index()

debit_features = debit_transactions.groupby('prism_consumer_id').agg(
    total_debit=('amount', 'sum')
).reset_index()

credit_debit_features = credit_features.merge(debit_features, on='prism_consumer_id', how='outer').fillna(0)
consumer_features = consumer.merge(credit_debit_features, on='prism_consumer_id', how='left').fillna(0)

run_classification2(['total_credit', 'total_debit'], 'DQ_TARGET', consumer_features)

Logistic Regression for ['total_credit', 'total_debit']
              precision    recall  f1-score   support

         0.0       0.96      0.36      0.52      2784
         1.0       0.09      0.79      0.16       216

    accuracy                           0.39      3000
   macro avg       0.52      0.57      0.34      3000
weighted avg       0.89      0.39      0.49      3000

ROC-AUC Score: 0.604

Random Forest Classification for ['total_credit', 'total_debit']
              precision    recall  f1-score   support

         0.0       0.94      0.76      0.84      2784
         1.0       0.11      0.39      0.17       216

    accuracy                           0.73      3000
   macro avg       0.53      0.57      0.51      3000
weighted avg       0.88      0.73      0.79      3000

ROC-AUC Score: 0.600
[LightGBM] [Info] Number of positive: 11210, number of negative: 11210
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can

In [62]:
credit_features

,prism_consumer_id,total_credit
0,0,14386.82
1,1,24903.80
2,10,20576.56
3,100,35236.84
4,1000,78353.07
...,...,...
14471,9995,16925.84
14472,9996,1200.03
14473,9997,17206.11
14474,9998,14566.37


In [63]:
consumer_balance['DQ_TARGET'].value_counts()

DQ_TARGET
0.0    10994
1.0     1006
Name: count, dtype: int64

In [64]:
consumer_balance.isna().sum()

prism_consumer_id       0
evaluation_date         0
credit_score            0
DQ_TARGET            3000
balance              1991
std_credit              0
std_balance          1991
dtype: int64

In [65]:
new_data = consumer_balance.dropna()
new_data['DQ_TARGET'].value_counts()

DQ_TARGET
0.0    9493
1.0     915
Name: count, dtype: int64

In [116]:
import pandas as pd

transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])

# sort transactions by consumer ID and date
transactions = transactions.sort_values(by=['prism_consumer_id', 'posted_date'])

def calculate_balances(group):
    balance = 0
    balances = []  # cumulative balances
    for _, row in group.iterrows():
        if row['credit_or_debit'] == 'CREDIT':
            balance += row['amount']  # add for CREDIT
        elif row['credit_or_debit'] == 'DEBIT':
            balance -= row['amount']  # subtract for DEBIT
        balances.append(balance)
    group['calculated_balance'] = balances
    return group

transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances)

transactions_with_balances.head()


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/2405563956.py:20: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances)


prism_consumer_id prism_transaction_id  category  \
prism_consumer_id                                                           
0                 136802                 0               136738        14   
                  136767                 0               136703        11   
                  136803                 0               136739        39   
                  136804                 0               136740        37   
                  136805                 0               136741        14   

                           amount credit_or_debit posted_date  days_since  \
prism_consumer_id                                                           
0                 136802    27.62           DEBIT  2021-03-16        1147   
                  136767  1400.00          CREDIT  2021-03-17        1146   
                  136803    25.10           DEBIT  2021-03-17        1146   
                  136804   500.00           DEBIT  2021-03-17        1146   
                  136805    25.00           DEBIT  2021-03-18        1145   

                          calculated_balance  
prism_consumer_id                             
0                 136802              -27.62  
                  136767             1372.38  
                  136803             1347.28  
                  136804              847.28  
                  136805              822.28

In [118]:
# create new features

transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])

transactions = transactions.sort_values(by=['prism_consumer_id', 'posted_date'])

def calculate_balances(group):
    balance = 0
    balances = []
    for _, row in group.iterrows():
        if row['credit_or_debit'] == 'CREDIT':
            balance += row['amount']
        elif row['credit_or_debit'] == 'DEBIT':
            balance -= row['amount']
        balances.append(balance)
    group['calculated_balance'] = balances
    return group

transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances).reset_index(drop=True)

transactions_with_balances['days_since'] = (transactions_with_balances['posted_date'].max() - transactions_with_balances['posted_date']).dt.days

# time windows
time_windows = [30, 90, 180, 365, 730, 1095]  # 1 month, 3 months, 6 months, 1 year, 2 years, 3 years

# groupby prism_consumer_id and calculate features
transaction_features = transactions_with_balances.groupby('prism_consumer_id').apply(lambda group: {
    # total credit and debit
    'total_credit': group[group['credit_or_debit'] == 'CREDIT']['amount'].sum(),
    'total_debit': group[group['credit_or_debit'] == 'DEBIT']['amount'].sum(),
    
    # spending in different time windows
    **{f'spending_{tw}d': group[group['days_since'] <= tw]['amount'].sum() for tw in time_windows},
    
    # spending based on categories (ex. 49 categories)
    **{f'category_{cat}_spending': group[group['category'] == cat]['amount'].sum() for cat in transactions_with_balances['category'].unique()},
    
    # balance-based features
    'max_balance': group['calculated_balance'].max(),
    'min_balance': group['calculated_balance'].min(),
    'mean_balance': group['calculated_balance'].mean(),
    'std_balance': group['calculated_balance'].std(),
    
    # number of times balance was negative
    'negative_balance_count': (group['calculated_balance'] < 0).sum(),
    
    # duration of account (days)
    'account_duration_days': group['days_since'].max()
}).apply(pd.Series).reset_index()

# merge transaction features with consumer data
consumer_features = consumer.merge(transaction_features, on='prism_consumer_id', how='left')

# fill NaN values with 0 (ex. consumers with no transactions)
consumer_features.fillna(0, inplace=True)

print(consumer_features.head())


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/990600752.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances).reset_index(drop=True)
/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/990600752.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transaction_features = transaction

  prism_consumer_id evaluation_date  credit_score  DQ_TARGET  total_credit  \
0                 0      2021-09-01         726.0        0.0      14386.82   
1                 1      2021-07-01         626.0        0.0      24903.80   
2                 2      2021-05-01         680.0        0.0      22764.71   
3                 3      2021-03-01         734.0        0.0      22641.25   
4                 4      2021-10-01         676.0        0.0      14966.11   

   total_debit  spending_30d  spending_90d  spending_180d  spending_365d  ...  \
0     14908.41           0.0           0.0            0.0            0.0  ...   
1     23098.37           0.0           0.0            0.0            0.0  ...   
2     22334.58           0.0           0.0            0.0            0.0  ...   
3     19846.01           0.0           0.0            0.0            0.0  ...   
4     17509.71           0.0           0.0            0.0            0.0  ...   

   category_43_spending  category_49_spendin

In [129]:
# test + store results of features

results = []

for feature in feature_columns:
    print(f"--- Testing Single Feature: {feature} ---")
    
    # features (X) and target (y)
    X = dataset[[feature]]
    y = dataset[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # balance the training data using SMOTE
    sm = SMOTE(random_state=42)
    X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

    # Logistic Regression
    log_model = LogisticRegression(class_weight='balanced', max_iter=1000)
    log_model.fit(X_train_bal, y_train_bal)
    log_y_proba = log_model.predict_proba(X_test)[:, 1]
    log_auc = roc_auc_score(y_test, log_y_proba)
    print(f"Logistic Regression ROC-AUC Score: {log_auc:.3f}")

    # Random Forest
    rfc_model = RandomForestClassifier(random_state=42)
    rfc_model.fit(X_train_bal, y_train_bal)
    rfc_y_proba = rfc_model.predict_proba(X_test)[:, 1]
    rfc_auc = roc_auc_score(y_test, rfc_y_proba)
    print(f"Random Forest ROC-AUC Score: {rfc_auc:.3f}")

    # LightGBM
    lgb_model = lgb.LGBMClassifier(random_state=42)
    lgb_model.fit(X_train_bal, y_train_bal)
    lgb_y_proba = lgb_model.predict_proba(X_test)[:, 1]
    lgb_auc = roc_auc_score(y_test, lgb_y_proba)
    print(f"LightGBM ROC-AUC Score: {lgb_auc:.3f}")

    # XGBoost
    xgb_model = xgb.XGBClassifier(scale_pos_weight=len(y_train) / y_train.sum(), random_state=42)
    xgb_model.fit(X_train_bal, y_train_bal)
    xgb_y_proba = xgb_model.predict_proba(X_test)[:, 1]
    xgb_auc = roc_auc_score(y_test, xgb_y_proba)
    print(f"XGBoost ROC-AUC Score: {xgb_auc:.3f}")

    # Append the results to the dictionary
    results.append({
        'feature': feature,
        'logistic_auc': log_auc,
        'random_forest_auc': rfc_auc,
        'lightgbm_auc': lgb_auc,
        'xgboost_auc': xgb_auc
    })

    print("\n")

results_df = pd.DataFrame(results)

# sort by best ROC-AUC score across models
results_df['best_auc'] = results_df[['logistic_auc', 'random_forest_auc', 'lightgbm_auc', 'xgboost_auc']].max(axis=1)
results_df = results_df.sort_values(by='best_auc', ascending=False)

print("Top Features by ROC-AUC:")
print(results_df[['feature', 'best_auc']].head(10))


--- Testing Single Feature: total_credit ---
Logistic Regression ROC-AUC Score: 0.603
Random Forest ROC-AUC Score: 0.543
[LightGBM] [Info] Number of positive: 11196, number of negative: 11196
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 22392, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LightGBM ROC-AUC Score: 0.563
XGBoost ROC-AUC Score: 0.552


--- Testing Single Feature: total_debit ---
Logistic Regression ROC-AUC Score: 0.589
Random Forest ROC-AUC Score: 0.552
[LightGBM] [Info] Number of positive: 11196, number of negative: 11196
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins

In [122]:
# group balances as a list for each consumer
grouped_balances = transactions_with_balances.groupby('prism_consumer_id')['calculated_balance'].apply(list).reset_index()
grouped_balances.rename(columns={'calculated_balance': 'all_balances'}, inplace=True)
print(grouped_balances.head())


  prism_consumer_id                                       all_balances
0                 0  [-27.62, 1372.38, 1347.2800000000002, 847.2800...
1                 1  [200.0, 289.0, 1096.28, 1007.28, 973.4, 773.4,...
2                10  [-18.15, -77.72999999999999, -191.49, -208.0, ...
3               100  [200.0, 0.0, -901.7, -1387.42, -1803.580000000...
4              1000  [500.0, 1200.0, 700.0, 381.92, -118.0799999999...


In [123]:
# Count how many times the calculated balance was negative for each consumer
negative_balance_count = transactions_with_balances.groupby('prism_consumer_id').apply(
    lambda group: (group['calculated_balance'] < 0).sum()
).reset_index()

negative_balance_count.columns = ['prism_consumer_id', 'negative_balance_count']
print(negative_balance_count.head())

  prism_consumer_id  negative_balance_count
0                 0                     237
1                 1                      27
2                10                     322
3               100                     119
4              1000                       8


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/465156714.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  negative_balance_count = transactions_with_balances.groupby('prism_consumer_id').apply(


In [125]:
# Calculate the account age in days

acct['balance_date'] = pd.to_datetime(acct['balance_date'])
acct['account_age'] = (acct['balance_date'].max() - acct['balance_date']).dt.days
account_age = acct.groupby('prism_consumer_id')['account_age'].max().reset_index()
account_age.columns = ['prism_consumer_id', 'account_age_days']
print(account_age)


      prism_consumer_id  account_age_days
0                     0               979
1                     1              1041
2                    10               826
3                   100              1102
4                  1000               461
...                 ...               ...
13004              9995               431
13005              9996               208
13006              9997               347
13007              9998               186
13008              9999               271

[13009 rows x 2 columns]


In [128]:
acct['balance'] = pd.to_numeric(acct['balance'], errors='coerce')

negative_balances = acct[acct['balance'] < 0]
if negative_balances.empty:
    print("No negative balances found in the dataset.")
else:
    print(negative_balances)


      prism_consumer_id prism_account_id account_type balance_date  balance  \
82                 2246               82     CHECKING   2021-04-30  -168.90   
206                2751              206     CHECKING   2022-03-30  -474.41   
506                2242              506     CHECKING   2022-03-31   -49.52   
616                3280              616     CHECKING   2021-11-30    -7.23   
904                1056              904     CHECKING   2021-03-31  -313.61   
...                 ...              ...          ...          ...      ...   
24239             12997            24239  CREDIT CARD   2021-12-14   -25.00   
24279             13976            24279  CREDIT CARD   2022-02-15 -1025.27   
24296             11678            24296     CHECKING   2021-12-19  -145.00   
24357             10353            24357      SAVINGS   2021-12-16   -10.00   
24426             10580            24426     CHECKING   2022-03-10   -48.69   

       total_balance    month  days_account_open  a

In [133]:
# Check if the balance is negative for each transaction
transactions_with_balances['is_negative'] = transactions_with_balances['calculated_balance'] < 0

# Group by prism_consumer_id and calculate:
# 1. The count of negative balances
# 2. The total number of transactions
negative_balance_ratio = transactions_with_balances.groupby('prism_consumer_id').agg(
    negative_count=('is_negative', 'sum'),  # Count of negative balances
    total_transactions=('calculated_balance', 'size')  # Total number of transactions
).reset_index()

# Calculate the ratio of negative balances to total transactions
negative_balance_ratio['negative_balance_ratio'] = (
    negative_balance_ratio['negative_count'] / negative_balance_ratio['total_transactions']
)

# Fill NaN values with 0 in case of division by zero or missing data
negative_balance_ratio.fillna(0, inplace=True)

# Output the result
print(negative_balance_ratio)

      prism_consumer_id  negative_count  total_transactions  \
0                     0             237                 408   
1                     1              27                 314   
2                    10             322                 341   
3                   100             119                 122   
4                  1000               8                 211   
...                 ...             ...                 ...   
14487              9995              95                 311   
14488              9996              35                  42   
14489              9997              71                 261   
14490              9998              59                 220   
14491              9999              25                 381   

       negative_balance_ratio  
0                    0.580882  
1                    0.085987  
2                    0.944282  
3                    0.975410  
4                    0.037915  
...                       ...  
14487              

In [134]:
feature_column = 'negative_balance_ratio'

X = consumer_features[[feature_column]]
y = consumer_features['DQ_TARGET']

run_classification2([feature_column], 'DQ_TARGET', consumer_features)


Logistic Regression for ['negative_balance_ratio']
              precision    recall  f1-score   support

         0.0       0.93      0.47      0.62      2784
         1.0       0.08      0.57      0.14       216

    accuracy                           0.48      3000
   macro avg       0.51      0.52      0.38      3000
weighted avg       0.87      0.48      0.59      3000

ROC-AUC Score: 0.533

Random Forest Classification for ['negative_balance_ratio']
              precision    recall  f1-score   support

         0.0       0.92      0.55      0.69      2784
         1.0       0.07      0.42      0.12       216

    accuracy                           0.54      3000
   macro avg       0.50      0.48      0.40      3000
weighted avg       0.86      0.54      0.65      3000

ROC-AUC Score: 0.488
[LightGBM] [Info] Number of positive: 11210, number of negative: 11210
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000508 seconds.
You can set `forc

In [137]:
debit_transactions = transactions[transactions['credit_or_debit'] == 'DEBIT']

# Calculate monthly and yearly outflows per consumer
outflow_features = debit_transactions.groupby('prism_consumer_id').apply(lambda group: {
    'avg_monthly_outflow': group['amount'].sum() / max(1, group['posted_date'].dt.to_period('M').nunique()),  # Avoid division by zero
    'avg_yearly_outflow': group['amount'].sum() / max(1, group['posted_date'].dt.to_period('Y').nunique())   # Avoid division by zero
}).apply(pd.Series).reset_index()

# Rename the columns for clarity
outflow_features.columns = ['prism_consumer_id', 'avg_monthly_outflow', 'avg_yearly_outflow']

# Display the resulting outflow features
print(outflow_features.head())



/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/4280441153.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  outflow_features = debit_transactions.groupby('prism_consumer_id').apply(lambda group: {


  prism_consumer_id  avg_monthly_outflow  avg_yearly_outflow
0                 0          2129.772857           14908.410
1                 1          3299.767143           23098.370
2                10          3109.514286           10883.300
3               100          6623.768333           19871.305
4              1000         11130.712857           38957.495


In [136]:
# Calculate total transactions (count and sum) by category
category_features = transactions.groupby(['prism_consumer_id', 'category']).apply(lambda group: {
    'total_transactions': len(group),
    'total_credit_amount': group[group['credit_or_debit'] == 'CREDIT']['amount'].sum(),
    'total_debit_amount': group[group['credit_or_debit'] == 'DEBIT']['amount'].sum(),
    'total_transaction_amount': group['amount'].sum()
}).apply(pd.Series).reset_index()


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/1327682312.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  category_features = transactions.groupby(['prism_consumer_id', 'category']).apply(lambda group: {


In [78]:
acct

,prism_consumer_id,prism_account_id,account_type,balance_date,balance,days_since
0,3023,0,SAVINGS,2021-08-31,90.57,979
1,3023,1,CHECKING,2021-08-31,225.95,979
2,4416,2,SAVINGS,2022-03-31,15157.17,767
3,4416,3,CHECKING,2022-03-31,66.42,767
4,4227,4,CHECKING,2021-07-31,7042.90,1010
...,...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75,771
24462,11615,24462,SAVINGS,2022-03-30,5.00,768
24463,11615,24463,CHECKING,2022-03-30,1956.46,768
24464,12210,24464,CHECKING,2022-03-28,2701.51,770


In [76]:
latest_balance

,prism_consumer_id,today_balance
0,0,25.70
1,0,294.67
2,1,3211.18
3,1,91.24
4,10,1.49
...,...,...
24087,9995,0.00
24088,9996,252.67
24089,9997,611.28
24090,9998,-862.99


In [77]:
balance_features

,prism_consumer_id,avg_balance_30d,avg_balance_90d,avg_balance_180d,avg_balance_365d,below_25_30d,below_25_90d,below_25_180d,below_25_365d,below_50_30d,below_50_90d,below_50_180d,below_50_365d,below_100_30d,below_100_90d,below_100_180d,below_100_365d,days_negative_balance
0,0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13004,9995,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13005,9996,0.0,0.0,0.0,252.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13006,9997,0.0,0.0,0.0,611.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13007,9998,0.0,0.0,0.0,-862.99,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [74]:
acct['balance_date'] = pd.to_datetime(acct['balance_date'], errors='coerce')
print(acct['balance_date'].isna().sum())  # Check for NaT


0


In [75]:
balance_features = acct.groupby('prism_consumer_id').apply(lambda group: {
    **{f'avg_balance_{tw}d': group[group['days_since'] <= tw]['balance'].mean() if not group[group['days_since'] <= tw].empty else 0 for tw in time_windows},
    **{f'below_{threshold}_{tw}d': (group[group['days_since'] <= tw]['balance'] < threshold).sum() for threshold in [25, 50, 100] for tw in time_windows},
    **{'days_negative_balance': (group['balance'] < 0).sum()}
}).apply(pd.Series).reset_index()


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/235641726.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balance_features = acct.groupby('prism_consumer_id').apply(lambda group: {


In [ ]:
missing_ids = set(consumer['prism_consumer_id']) - set(acct['prism_consumer_id'])
print(f"Missing IDs: {missing_ids}")


In [104]:
# Ensure dates are in datetime format
transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])
acct['balance_date'] = pd.to_datetime(acct['balance_date'])

# Step 1: Calculate cumulative daily balance for each consumer
transactions['amount_adjusted'] = transactions.apply(
    lambda x: x['amount'] if x['credit_or_debit'] == 'CREDIT' else -x['amount'], axis=1
)

# Group by consumer and date, calculate daily balance
daily_balance = transactions.groupby(['prism_consumer_id', 'posted_date'])['amount_adjusted'].sum().reset_index()
daily_balance['cumulative_balance'] = daily_balance.groupby('prism_consumer_id')['amount_adjusted'].cumsum()

# Step 2: Count negative balance days
negative_balance_days = daily_balance[daily_balance['cumulative_balance'] < 0].groupby('prism_consumer_id').size().reset_index(name='negative_balance_days')

# Step 3: Calculate account open duration (in days)
first_transaction_date = transactions.groupby('prism_consumer_id')['posted_date'].min().reset_index().rename(columns={'posted_date': 'first_transaction_date'})
first_transaction_date['days_open'] = (pd.Timestamp.today() - first_transaction_date['first_transaction_date']).dt.days

# Step 4: Merge negative balance days and account open duration
negative_ratio = negative_balance_days.merge(first_transaction_date, on='prism_consumer_id', how='left')
negative_ratio['negative_balance_ratio'] = negative_ratio['negative_balance_days'] / negative_ratio['days_open']

# Fill missing values for consumers with no negative balance days
negative_ratio[['negative_balance_days', 'negative_balance_ratio']].fillna(0, inplace=True)

# Step 5: Merge with the main consumer dataset
consumer = consumer.merge(negative_ratio[['prism_consumer_id', 'negative_balance_days', 'days_open', 'negative_balance_ratio']], on='prism_consumer_id', how='left')

# Fill NaN values with 0 for consumers with no transactions
consumer[['negative_balance_days', 'days_open', 'negative_balance_ratio']].fillna(0, inplace=True)

# Final Output
print(f"Consumer dataset with negative balance ratio and related columns added: {consumer.shape}")
consumer.head()


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/1935909462.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_ratio[['negative_balance_days', 'negative_balance_ratio']].fillna(0, inplace=True)


KeyError: "['negative_balance_days', 'negative_balance_ratio'] not in index"

In [105]:
negative_ratio

,prism_consumer_id,negative_balance_days,first_transaction_date,days_open,negative_balance_ratio
0,0,80,2021-03-16,1410,0.056738
1,1,10,2021-01-15,1470,0.006803
2,10,130,2021-08-17,1256,0.103503
3,100,54,2022-08-01,907,0.059537
4,1000,8,2022-08-31,877,0.009122
...,...,...,...,...,...
12781,9995,36,2022-12-05,781,0.046095
12782,9996,25,2023-07-21,553,0.045208
12783,9997,19,2023-02-27,697,0.027260
12784,9998,21,2023-08-07,536,0.039179


In [106]:
# Ensure dates are in datetime format
transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])
acct['balance_date'] = pd.to_datetime(acct['balance_date'])

# Step 1: Calculate cumulative daily balance for each consumer
transactions['amount_adjusted'] = transactions.apply(
    lambda x: x['amount'] if x['credit_or_debit'] == 'CREDIT' else -x['amount'], axis=1
)

# Group by consumer and date, calculate daily balance
daily_balance = transactions.groupby(['prism_consumer_id', 'posted_date'])['amount_adjusted'].sum().reset_index()
daily_balance['cumulative_balance'] = daily_balance.groupby('prism_consumer_id')['amount_adjusted'].cumsum()

# Step 2: Count negative balance days
negative_balance_days = daily_balance[daily_balance['cumulative_balance'] < 0].groupby('prism_consumer_id').size().reset_index(name='negative_balance_days')

# Step 3: Calculate account open duration (in days)
first_transaction_date = transactions.groupby('prism_consumer_id')['posted_date'].min().reset_index().rename(columns={'posted_date': 'first_transaction_date'})
first_transaction_date['days_open'] = (pd.Timestamp.today() - first_transaction_date['first_transaction_date']).dt.days

# Step 4: Merge negative balance days and account open duration
# Left join to ensure all consumers are included
negative_ratio = first_transaction_date.merge(negative_balance_days, on='prism_consumer_id', how='left')

# Fill missing values for consumers with no negative balance days
negative_ratio['negative_balance_days'] = negative_ratio['negative_balance_days'].fillna(0)

# Calculate the ratio of negative balance days to account open duration
negative_ratio['negative_balance_ratio'] = negative_ratio['negative_balance_days'] / negative_ratio['days_open']

# Step 5: Merge with the main consumer dataset
consumer = consumer.merge(negative_ratio[['prism_consumer_id', 'negative_balance_days', 'days_open', 'negative_balance_ratio']], on='prism_consumer_id', how='left')

# Fill NaN values for consumers with no transactions
consumer[['negative_balance_days', 'days_open', 'negative_balance_ratio']] = consumer[['negative_balance_days', 'days_open', 'negative_balance_ratio']].fillna(0)

# Final Output
print(f"Consumer dataset with negative balance ratio and related columns added: {consumer.shape}")
consumer.head()


KeyError: "['days_open'] not in index"

In [109]:
import pandas as pd

# Ensure 'posted_date' is in datetime format for proper sorting
transactions['posted_date'] = pd.to_datetime(transactions['posted_date'])

# Sort transactions by consumer ID and date
transactions = transactions.sort_values(by=['prism_consumer_id', 'posted_date'])

# Function to calculate balances
def calculate_balances(group):
    # Start with an initial balance of 0
    balance = 0
    balances = []  # List to hold cumulative balances
    for _, row in group.iterrows():
        if row['credit_or_debit'] == 'CREDIT':
            balance += row['amount']  # Add for CREDIT
        elif row['credit_or_debit'] == 'DEBIT':
            balance -= row['amount']  # Subtract for DEBIT
        balances.append(balance)
    group['calculated_balance'] = balances
    return group

# Apply the function to each consumer group
transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances)

# Display the resulting DataFrame
transactions_with_balances


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/3584599517.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transactions_with_balances = transactions.groupby('prism_consumer_id').apply(calculate_balances)


prism_consumer_id prism_transaction_id  category  \
prism_consumer_id                                                            
0                 136802                  0               136738        14   
                  136767                  0               136703        11   
                  136803                  0               136739        39   
                  136804                  0               136740        37   
                  136805                  0               136741        14   
...                                     ...                  ...       ...   
9999              1524647              9999              1522635        16   
                  1524648              9999              1522636        14   
                  1524649              9999              1522637        14   
                  1524650              9999              1522638        16   
                  1524651              9999              1522639        16   

                            amount credit_or_debit posted_date  inflow  \
prism_consumer_id                                                        
0                 136802     27.62           DEBIT  2021-03-16   False   
                  136767   1400.00          CREDIT  2021-03-17    True   
                  136803     25.10           DEBIT  2021-03-17   False   
                  136804    500.00           DEBIT  2021-03-17   False   
                  136805     25.00           DEBIT  2021-03-18   False   
...                            ...             ...         ...     ...   
9999              1524647    66.63           DEBIT  2023-08-08   False   
                  1524648    16.91           DEBIT  2023-08-08   False   
                  1524649     3.52           DEBIT  2023-08-08   False   
                  1524650     7.99           DEBIT  2023-08-08   False   
                  1524651    16.99           DEBIT  2023-08-08   False   

                           outflow  is_negative_balance  is_high_balance  \
prism_consumer_id                                                          
0                 136802      True                False            False   
                  136767     False                False             True   
                  136803      True                False            False   
                  136804      True                False            False   
                  136805      True                False            False   
...                            ...                  ...              ...   
9999              1524647     True                False            False   
                  1524648     True                False            False   
                  1524649     True                False            False   
                  1524650     True                False            False   
                  1524651     True                False            False   

                             month  amount_adjusted  calculated_balance  
prism_consumer_id                                                        
0                 136802   2021-03           -27.62              -27.62  
                  136767   2021-03          1400.00             1372.38  
                  136803   2021-03           -25.10             1347.28  
                  136804   2021-03          -500.00              847.28  
                  136805   2021-03           -25.00              822.28  
...                            ...              ...                 ...  
9999              1524647  2023-08           -66.63             -274.02  
                  1524648  2023-08           -16.91             -290.93  
                  1524649  2023-08            -3.52             -294.45  
                  1524650  2023-08            -7.99             -302.44  
                  1524651  2023-08           -16.99             -319.43  

[6407321 rows x 13 columns]

In [111]:
import pandas as pd
import numpy as np

# Ensure posted_date is in datetime format
transactions['posted_date'] = pd.to_datetime(transactions['posted_date'], errors='coerce')

# Calculate days since the most recent transaction for each consumer
transactions['days_since'] = (transactions['posted_date'].max() - transactions['posted_date']).dt.days

# Define time windows for spending behavior
time_windows = [0, 350, 700, 1050, 1401]

# Feature engineering based on transactions
transaction_features = transactions.groupby('prism_consumer_id').apply(lambda group: {
    # Total inflow (CREDIT) and outflow (DEBIT)
    'total_credit': group[group['credit_or_debit'] == 'CREDIT']['amount'].sum(),
    'total_debit': group[group['credit_or_debit'] == 'DEBIT']['amount'].sum(),
    # Net transaction balance
    'net_transaction_balance': group[group['credit_or_debit'] == 'CREDIT']['amount'].sum() - group[group['credit_or_debit'] == 'DEBIT']['amount'].sum(),
    # Spending in different time windows
    **{f'spending_last_{tw}_days': group[group['days_since'] <= tw]['amount'].sum() for tw in time_windows},
    # Spending grouped by categories
    **{f'category_{cat}_spending': group[group['category'] == cat]['amount'].sum() for cat in transactions['category'].unique()},
}).apply(pd.Series).reset_index()

# Merge transaction features with consumer data
consumer_features = consumer.merge(transaction_features, on='prism_consumer_id', how='left')

# Fill NaN values with 0 for consumers with no transactions
consumer_features.fillna(0, inplace=True)

# Check the result
print(consumer_features.head())

# Group by 'prism_consumer_id' and aggregate balance information
grouped_balances = acct.groupby('prism_consumer_id').agg({
    # List of all balances
    'balance': lambda x: list(x),
    # Count of negative balances
    'balance': lambda x: (np.array(x) < 0).sum(),
    # Average balance
    'balance': 'mean',
    # Max balance
    'balance': 'max',
    # Min balance
    'balance': 'min'
}).reset_index()

# Rename columns for clarity
grouped_balances.rename(columns={
    'balance': 'all_balances',
    '<lambda_0>': 'negative_balance_count',
    '<lambda_1>': 'average_balance',
    '<lambda_2>': 'max_balance',
    '<lambda_3>': 'min_balance'
}, inplace=True)

print(grouped_balances.head())

# Calculate how long each account has been open
acct['balance_date'] = pd.to_datetime(acct['balance_date'], errors='coerce')
acct['days_account_open'] = (acct['balance_date'].max() - acct['balance_date']).dt.days

# Add ratio of negative balance count to account age
negative_balance_ratio = acct.groupby('prism_consumer_id').apply(lambda group: {
    'negative_balance_days': (group['balance'] < 0).sum(),
    'days_account_open': (group['balance_date'].max() - group['balance_date'].min()).days,
    'negative_balance_ratio': (group['balance'] < 0).sum() / ((group['balance_date'].max() - group['balance_date'].min()).days + 1)
}).apply(pd.Series).reset_index()

print(negative_balance_ratio.head())

# Merge grouped balances and negative balance ratio with consumer features
final_consumer_features = consumer_features.merge(grouped_balances, on='prism_consumer_id', how='left')
final_consumer_features = final_consumer_features.merge(negative_balance_ratio, on='prism_consumer_id', how='left')

# Fill NaN values with 0
final_consumer_features.fillna(0, inplace=True)

# Final dataframe
print(final_consumer_features.head())


/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/938016773.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  transaction_features = transactions.groupby('prism_consumer_id').apply(lambda group: {


  prism_consumer_id evaluation_date  credit_score  DQ_TARGET  total_balance  \
0                 0      2021-09-01         726.0        0.0         320.37   
1                 1      2021-07-01         626.0        0.0        3302.42   
2                 2      2021-05-01         680.0        0.0        2805.36   
3                 3      2021-03-01         734.0        0.0        7667.01   
4                 4      2021-10-01         676.0        0.0         394.55   

   latest_balance  avg_balance_12m  avg_balance_9m  avg_balance_6m  \
0           25.70              0.0             0.0             0.0   
1         3211.18              0.0             0.0             0.0   
2         2561.43              0.0             0.0             0.0   
3         6690.19              0.0             0.0             0.0   
4          391.62              0.0             0.0             0.0   

   days_since_open  ...  category_32_spending  category_48_spending  \
0           1242.0  ...          

/var/folders/4d/mwcq9vv51rscfd92n_2xs8vc0000gn/T/ipykernel_11723/938016773.py:65: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  negative_balance_ratio = acct.groupby('prism_consumer_id').apply(lambda group: {


  prism_consumer_id  negative_balance_days  days_account_open  \
0                 0                    0.0                0.0   
1                 1                    0.0                0.0   
2                10                    0.0                0.0   
3               100                    0.0              610.0   
4              1000                    0.0                0.0   

   negative_balance_ratio  
0                     0.0  
1                     0.0  
2                     0.0  
3                     0.0  
4                     0.0  


MergeError: Passing 'suffixes' which cause duplicate columns {'negative_balance_days_x', 'negative_balance_ratio_x'} is not allowed.